This concept notebook is to showcase the underlying machinery that will be used for Imviz image rotation front-end in the future.

In [ ]:
import gwcs
import numpy as np
from astropy import units as u
from astropy.coordinates import ICRS
from astropy.modeling import models
from astropy.nddata import NDData
from astropy.wcs import WCS
from gwcs import coordinate_frames as cf

from jdaviz import Imviz
from jdaviz.configs.imviz.wcs_utils import get_compass_info, _get_rotated_nddata_from_label

These data are from `BaseImviz_WCS_GWCS` test class.

Image without any WCS.

In [ ]:
np.random.seed(42)
arr = np.random.random((10, 8))  # (ny, nx)
arr[0, 0] = 10  # Bright corner for sanity check

FITS WCS.

In [ ]:
w_fits = WCS({'WCSAXES': 2, 'NAXIS1': 8, 'NAXIS2': 10,
              'CRPIX1': 5.0, 'CRPIX2': 5.0,
              'PC1_1': -1.14852e-05, 'PC1_2': 7.01477e-06,
              'PC2_1': 7.75765e-06, 'PC2_2': 1.20927e-05,
              'CDELT1': 1.0, 'CDELT2': 1.0,
              'CUNIT1': 'deg', 'CUNIT2': 'deg',
              'CTYPE1': 'RA---TAN', 'CTYPE2': 'DEC--TAN',
              'CRVAL1': 3.581704851882, 'CRVAL2': -30.39197867265,
              'LONPOLE': 180.0, 'LATPOLE': -30.39197867265,
              'MJDREF': 0.0, 'RADESYS': 'ICRS'})

GWCS.

In [ ]:
shift_by_crpix = models.Shift(-(5 - 1) * u.pix) & models.Shift(-(5 - 1) * u.pix)
matrix = np.array([[1.290551569736E-05, 5.9525007864732E-06],
                   [5.0226382102765E-06, -1.2644844123757E-05]])
rotation = models.AffineTransformation2D(matrix * u.deg, translation=[0, 0] * u.deg)
rotation.input_units_equivalencies = {"x": u.pixel_scale(1 * (u.deg / u.pix)),
                                      "y": u.pixel_scale(1 * (u.deg / u.pix))}
rotation.inverse = models.AffineTransformation2D(np.linalg.inv(matrix) * u.pix,
                                                 translation=[0, 0] * u.pix)
rotation.inverse.input_units_equivalencies = {"x": u.pixel_scale(1 * (u.pix / u.deg)),
                                              "y": u.pixel_scale(1 * (u.pix / u.deg))}
tan = models.Pix2Sky_TAN()
celestial_rotation = models.RotateNative2Celestial(
    3.581704851882 * u.deg, -30.39197867265 * u.deg, 180 * u.deg)
det2sky = shift_by_crpix | rotation | tan | celestial_rotation
det2sky.name = "linear_transform"
detector_frame = cf.Frame2D(name="detector", axes_names=("x", "y"), unit=(u.pix, u.pix))
sky_frame = cf.CelestialFrame(reference_frame=ICRS(), name='icrs', unit=(u.deg, u.deg))
pipeline = [(detector_frame, det2sky), (sky_frame, None)]
w_gwcs = gwcs.WCS(pipeline)
w_gwcs.bounding_box = ((0, 8), (0, 10)) * u.pix  # x, y

Load data into Imviz:

1. Data with FITS WCS and unit.
2. Data with GWCS (rotated w.r.t. FITS WCS) and no unit.
3. Data without WCS nor unit.

In [ ]:
imviz = Imviz()
imviz.load_data(NDData(arr, wcs=w_fits, unit='electron/s'), data_label='fits_wcs')
imviz.load_data(NDData(arr, wcs=w_gwcs), data_label='gwcs')
imviz.load_data(arr, data_label='no_wcs')
imviz.show()

In [ ]:
imviz.default_viewer.zoom_level = "fit"

Open up the Compass plugin to see where the celestial axes are, if any.

Let's say we want N-up E-left orientation. We generate a fake data with the desired orientation.

In [ ]:
data = imviz.default_viewer.state.reference_data
print(data.label)

In [ ]:
degn = get_compass_info(data.coords, data.shape)[0] * u.deg
print(degn)

In [ ]:
# FIXME: Is degn supposed to be pass-in as-is? How to make it N-up E-left?
# For the data in this notebook, pixels in viewer is flipped left-right compared
# to what you would expect from Compass plugin.
# Maybe we need some hardcoded WCS for N-up E-left/right instead of using _rotated_gwcs()
fake_ndd_rotated = _get_rotated_nddata_from_label(imviz.app, data.label, degn)

Once we have made the Data object with the desired WCS, we can add it to the collection and also the viewer, but do not display it.

In [ ]:
imviz.load_data(fake_ndd_rotated, data_label=imviz.app._wcs_only_label)

Then, we make this Data object with the desired WCS a reference data. When link type is "pixels", you should not see any difference (no-op).

In [ ]:
imviz.app._change_reference_data(imviz.app._wcs_only_label)
print(imviz.default_viewer.state.reference_data.label)

If you want the other data with WCS to follow the orientation of this desired WCS, change the link type to "wcs". For data without any WCS, they will appear very weird because they are now linked by pixels to a Data with no real pixels.

Due to the nature of linking, you have to reset the zoom to see the data again.

(Dev note: Attempts to skip over Data without WCS for this step failed.)

In [ ]:
imviz.link_data(link_type="wcs")
imviz.default_viewer.zoom_level = "fit"

You can run the following cell any time to inspect the current state of Imviz linking.

In [ ]:
for elink in imviz.app.data_collection.external_links:
    elink_labels = (elink.data1.label, elink.data2.label)
    print(elink_labels, elink.__class__.__name__, elink.cids1, elink.cids2)

Changing back to pixel linking should look as before we link with WCS.

In [ ]:
imviz.link_data(link_type="pixels")
imviz.default_viewer.zoom_level = "fit"

Changing back to WCS linking and switching the reference data back to real data should look as if the fake data with WCS was never there.

In [ ]:
imviz.link_data(link_type="wcs")
imviz.app._change_reference_data("fits_wcs[DATA]")

In the multi-viewer case, the second viewer is allowed to have a different reference data than the default viewer.

Additionally, the "no_wcs" case acts weird while blinking in the default viewer while "_WCS_ONLY" is set as the reference data in the default viewer and things are linked by WCS; i.e., the Compass would show you are seeing "no_wcs" but it no longer disappears from view (it disappeared in the single-viewer case above).

(Dev note: Should we add a warning against using this kind of rotation with data without WCS and/or mult-viewer setup?)

In [ ]:
viewer_1 = imviz.create_image_viewer()

In [ ]:
imviz.app.add_data_to_viewer("imviz-1", "fits_wcs[DATA]")

In [ ]:
imviz.app._change_reference_data(imviz.app._wcs_only_label)

In [ ]:
print(imviz.default_viewer.state.reference_data.label)
print(viewer_1.state.reference_data.label)

Unlike the demo at https://gist.github.com/bmorris3/ee3af2e096fc869899280d645bb1b914, you would find it impossible to rotate the same data at two different angles at once. The best you could hope for is one viewer rotates the data to the desired WCS and the second viewer shows you the original data orientation. This is because that Gist was loading data at different angles as different data entries, which we would not do in production. (Though maybe this can be disproven as this feature is developed more; not sure.)

(Dev note: To avoid eating up precious links, when user wants a new rotation angle, we will overwrite the WCS in this fake data and re-link, if that is possible.)

(Dev note: After this part, sometimes the viewer display looks very elongated though not always, but the Compass display seems fine. Not sure why.)

In [ ]:
fake_ndd_rotated_2 = _get_rotated_nddata_from_label(imviz.app, "fits_wcs[DATA]", -90 * u.deg)

In [ ]:
imviz.app.data_collection[imviz.app._wcs_only_label].coords = fake_ndd_rotated_2.wcs
imviz.link_data(link_type=imviz.app._link_type, wcs_use_affine=imviz.app._wcs_use_affine, error_on_fail=True)

In [ ]:
imviz.default_viewer.zoom_level = "fit"